# Product manager - Data science challenge manager

https://nuwe.io/dev/train/hubs/nuwe/nuwe-open-positions/data-science-challenge-manager

In [1]:
# Data Processing
import pandas as pd
import numpy as np

# Modelling
from sklearn.preprocessing import OrdinalEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split


In [2]:
Y_COL = "Attrition_Flag"
TRAIN_FILE = "supply_chain_train.csv"
TEST_FILE = "supply_chain_test.csv"


In [3]:
df = pd.read_csv(TRAIN_FILE)

# CLIENTNUM (id) should not be used for training
df = df.drop(columns=["train_idx", "CLIENTNUM"])

print(df.shape)
df.head()


(8101, 20)


,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income_Category,Card_Category,Months_on_book,Total_Relationship_Count,Months_Inactive_12_mon,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio,Attrition_Flag
0,54,F,1,Unknown,Single,Unknown,Blue,36,1,3,3,3723.0,1728,1995.0,0.595,8554,99,0.678,0.464,1
1,58,F,4,High School,Married,Unknown,Blue,48,1,4,3,5396.0,1803,3593.0,0.493,2107,39,0.393,0.334,0
2,45,F,4,Unknown,Single,Less than $40K,Gold,36,6,1,3,15987.0,1648,14339.0,0.732,1436,36,1.250,0.103,1
3,34,F,2,Graduate,Single,Less than $40K,Blue,36,4,3,4,3625.0,2517,1108.0,1.158,2616,46,1.300,0.694,1
4,49,F,2,High School,Married,$40K - $60K,Blue,39,5,3,4,2720.0,1926,794.0,0.602,3806,61,0.794,0.708,1


In [4]:
df.dtypes


Customer_Age                  int64
Gender                       object
Dependent_count               int64
Education_Level              object
Marital_Status               object
Income_Category              object
Card_Category                object
Months_on_book                int64
Total_Relationship_Count      int64
Months_Inactive_12_mon        int64
Contacts_Count_12_mon         int64
Credit_Limit                float64
Total_Revolving_Bal           int64
Avg_Open_To_Buy             float64
Total_Amt_Chng_Q4_Q1        float64
Total_Trans_Amt               int64
Total_Trans_Ct                int64
Total_Ct_Chng_Q4_Q1         float64
Avg_Utilization_Ratio       float64
Attrition_Flag                int64
dtype: object

## Encoding categorical variables

In [5]:
text_cols = df.loc[:, df.dtypes == object].columns
df.loc[:4, text_cols]


,Gender,Education_Level,Marital_Status,Income_Category,Card_Category
0,F,Unknown,Single,Unknown,Blue
1,F,High School,Married,Unknown,Blue
2,F,Unknown,Single,Less than $40K,Gold
3,F,Graduate,Single,Less than $40K,Blue
4,F,High School,Married,$40K - $60K,Blue


In [6]:
for text_col in text_cols:
    print(f"-- {text_col} --")
    print(df[text_col].value_counts().to_string())
    print()


-- Gender --
F    4279
M    3822

-- Education_Level --
Graduate         2528
High School      1619
Unknown          1205
Uneducated       1171
College           816
Post-Graduate     407
Doctorate         355

-- Marital_Status --
Married     3767
Single      3144
Divorced     611
Unknown      579

-- Income_Category --
Less than $40K    2812
$40K - $60K       1453
$80K - $120K      1237
$60K - $80K       1122
Unknown            889
$120K +            588

-- Card_Category --
Blue        7557
Silver       436
Gold          93
Platinum      15



In [7]:
# One-hot: Gender, Marital_Status
# Ordinal Encoding: Education_Level, Income_Category, Card_Category

# Contain unknowns: Education_Level, Marital_Status, Income_Category


In [8]:
def one_hot_encoding(df: pd.DataFrame) -> pd.DataFrame:
    # One-hot encoding "Gender"
    return pd.concat([df, pd.get_dummies(df["Gender"])], axis=1)


In [9]:
card_category = ["Blue", "Silver", "Gold", "Platinum"]

income_category = [
    "Less than $40K",
    "$40K - $60K",
    "$80K - $120K",
    "$60K - $80K",
    "$120K +",
    "Unknown",
]

education_category = [
    "Uneducated",
    "High School",
    "College",
    "Graduate",
    "Post-Graduate",
    "Doctorate",
    "Unknown",
]


In [10]:
# Categorical encoding the three columns
ordinals = {
    "Card_Category": card_category,
    "Income_Category": income_category,
    "Education_Level": education_category,
}


def ordinal_encoding(df: pd.DataFrame) -> pd.DataFrame:
    for col_name, categories in ordinals.items():
        encoder = OrdinalEncoder(categories=[categories])
        df[col_name] = encoder.fit_transform(df[[col_name]])

    # Remove "Unknown" from the labels
    df["Education_Level"] = df["Education_Level"].replace(
        float(len(education_category) - 1), np.NaN
    )
    df["Income_Category"] = df["Income_Category"].replace(
        float(len(income_category) - 1), np.NaN
    )

    return df


In [11]:
def perform_encodings(df: pd.DataFrame) -> pd.DataFrame:
    one_hot_encoded = one_hot_encoding(df)
    return ordinal_encoding(one_hot_encoded)


In [12]:
df = perform_encodings(df)
df.loc[:4, ["F", "M", *text_cols]]


,F,M,Gender,Education_Level,Marital_Status,Income_Category,Card_Category
0,1,0,F,NaN,Single,NaN,0.0
1,1,0,F,1.0,Married,NaN,0.0
2,1,0,F,NaN,Single,0.0,2.0
3,1,0,F,3.0,Single,0.0,0.0
4,1,0,F,1.0,Married,1.0,0.0


In [13]:
df = df.drop(columns=["Gender"])

# Not having too much time to deal with "Martial_Status"
# Ideally, it would be one-hot encoded and studied the NaNs
df = df.drop(columns=["Marital_Status"])


In [14]:
# No outliers hav been detected
df.describe()


,Customer_Age,Dependent_count,Education_Level,Income_Category,Card_Category,Months_on_book,Total_Relationship_Count,Months_Inactive_12_mon,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio,Attrition_Flag,F,M
count,8101.000000,8101.000000,6896.000000,7212.000000,8101.000000,8101.000000,8101.000000,8101.000000,8101.000000,8101.000000,8101.000000,8101.000000,8101.000000,8101.000000,8101.000000,8101.000000,8101.000000,8101.000000,8101.000000,8101.000000
mean,46.306382,2.334773,2.064675,1.337354,0.082336,35.923590,3.813233,2.346871,2.450315,8636.548068,1160.382792,7476.165276,0.760809,4402.988150,64.907789,0.712176,0.273187,0.839526,0.528206,0.471794
std,8.022527,1.289564,1.405735,1.343109,0.331119,8.024359,1.551838,1.014177,1.100687,9086.419557,815.504293,9080.279910,0.216668,3401.709545,23.556379,0.239321,0.274595,0.367068,0.499235,0.499235
min,26.000000,0.000000,0.000000,0.000000,0.000000,13.000000,1.000000,0.000000,0.000000,1438.300000,0.000000,3.000000,0.000000,510.000000,10.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,41.000000,1.000000,1.000000,0.000000,0.000000,31.000000,3.000000,2.000000,2.000000,2555.000000,326.000000,1341.000000,0.632000,2160.000000,45.000000,0.583000,0.022000,1.000000,0.000000,0.000000
50%,46.000000,2.000000,2.000000,1.000000,0.000000,36.000000,4.000000,2.000000,2.000000,4549.000000,1273.000000,3495.000000,0.738000,3897.000000,67.000000,0.702000,0.174000,1.000000,1.000000,0.000000
75%,52.000000,3.000000,3.000000,2.000000,0.000000,40.000000,5.000000,3.000000,3.000000,11128.000000,1782.000000,9942.000000,0.859000,4739.000000,81.000000,0.818000,0.497000,1.000000,1.000000,1.000000
max,70.000000,5.000000,5.000000,4.000000,3.000000,56.000000,6.000000,6.000000,6.000000,34516.000000,2517.000000,34516.000000,2.675000,18484.000000,139.000000,3.714000,0.999000,1.000000,1.000000,1.000000


In [15]:
df.isna().sum(axis=1).astype(bool).sum()


1981

In [16]:
print(df.shape)
df2 = df.dropna()
print(df2.shape)


(8101, 20)
(6120, 20)


## Train the model

In [17]:
# Split the data into features (X) and target (y)
X = df2.drop(Y_COL, axis=1)
y = df2[Y_COL]

# Setting a random state, so that results can easily be replicated
RANDOM_STATE = 22

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=RANDOM_STATE
)


In [18]:
rf = RandomForestClassifier(random_state=RANDOM_STATE)
rf.fit(X_train, y_train)

y_pred = rf.predict(X_test)


## Train metrics

In [19]:
print(f"Accuracy:  {accuracy_score(y_test, y_pred):.4}")
print(f"Precision: {precision_score(y_test, y_pred):.4}")
print(f"Recall:    {recall_score(y_test, y_pred):.4}")
print(f"F1:        {f1_score(y_test, y_pred):.4}")


Accuracy:  0.9616
Precision: 0.9658
Recall:    0.9893
F1:        0.9774


In [20]:
confusion_matrix(y_test, y_pred)


array([[ 159,   36],
       [  11, 1018]], dtype=int64)

In [21]:
# As next steps, hyperparameters could be optimized
# However, there results are good enough


## Full prediction

In [22]:
# We can use full dataset as training data
rf = RandomForestClassifier(random_state=RANDOM_STATE)
rf.fit(X, y)

# RandomForest has been chosen because it is one of the easiest when it comes to explainability
# In a real-world scenario, a bank will surely want to know why each client stays or leaves


RandomForestClassifier(random_state=22)

In [23]:
predictions = pd.read_csv(TEST_FILE)
print(predictions.shape)
predictions.head()


(2026, 21)


,test_idx,CLIENTNUM,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income_Category,Card_Category,Months_on_book,...,Months_Inactive_12_mon,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio
0,0,719455083,48,F,3,Uneducated,Single,Less than $40K,Blue,39,...,3,4,2991.0,1508,1483.0,0.703,3734,64,0.882,0.504
1,1,773503308,59,M,1,Uneducated,Single,Less than $40K,Blue,53,...,5,4,2192.0,1569,623.0,0.706,4010,79,0.717,0.716
2,2,715452408,37,F,2,Graduate,Divorced,Less than $40K,Blue,36,...,3,3,1734.0,987,747.0,0.879,4727,67,0.914,0.569
3,3,711264033,47,M,3,Doctorate,Divorced,$40K - $60K,Blue,36,...,2,3,4786.0,1516,3270.0,0.940,4973,74,0.850,0.317
4,4,718943508,42,M,3,Unknown,Single,$80K - $120K,Blue,33,...,3,2,3714.0,2170,1544.0,0.524,1454,35,0.522,0.584


In [24]:
predictions.drop(columns=["test_idx", "CLIENTNUM"], inplace=True)
predictions = perform_encodings(predictions)
predictions.drop(columns=["Gender", "Marital_Status"], inplace=True)
predictions.head()


,Customer_Age,Dependent_count,Education_Level,Income_Category,Card_Category,Months_on_book,Total_Relationship_Count,Months_Inactive_12_mon,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio,F,M
0,48,3,0.0,0.0,0.0,39,4,3,4,2991.0,1508,1483.0,0.703,3734,64,0.882,0.504,1,0
1,59,1,0.0,0.0,0.0,53,5,5,4,2192.0,1569,623.0,0.706,4010,79,0.717,0.716,0,1
2,37,2,3.0,0.0,0.0,36,4,3,3,1734.0,987,747.0,0.879,4727,67,0.914,0.569,1,0
3,47,3,5.0,1.0,0.0,36,4,2,3,4786.0,1516,3270.0,0.940,4973,74,0.850,0.317,0,1
4,42,3,NaN,2.0,0.0,33,3,3,2,3714.0,2170,1544.0,0.524,1454,35,0.522,0.584,0,1


In [25]:
predictions.describe()


,Customer_Age,Dependent_count,Education_Level,Income_Category,Card_Category,Months_on_book,Total_Relationship_Count,Months_Inactive_12_mon,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio,F,M
count,2026.000000,2026.000000,1712.000000,1803.000000,2026.000000,2026.00000,2026.000000,2026.000000,2026.000000,2026.000000,2026.000000,2026.000000,2026.000000,2026.000000,2026.000000,2026.000000,2026.000000,2026.000000,2026.000000
mean,46.404245,2.391905,2.046729,1.291736,0.088845,35.94768,3.809970,2.318361,2.475321,8613.583021,1172.535538,7441.047483,0.756469,4408.477295,64.662389,0.712407,0.281719,0.532577,0.467423
std,7.995428,1.334967,1.441284,1.345966,0.344271,7.83479,1.565025,0.996204,1.128151,9100.417269,813.045248,9134.368529,0.229100,3379.585684,23.139140,0.233142,0.279998,0.499061,0.499061
min,26.000000,0.000000,0.000000,0.000000,0.000000,13.00000,1.000000,0.000000,0.000000,1438.300000,0.000000,14.000000,0.000000,530.000000,10.000000,0.000000,0.000000,0.000000,0.000000
25%,41.000000,1.000000,1.000000,0.000000,0.000000,32.00000,3.000000,2.000000,2.000000,2571.250000,504.750000,1255.000000,0.625000,2147.250000,45.000000,0.579000,0.026000,0.000000,0.000000
50%,46.000000,2.000000,2.000000,1.000000,0.000000,36.00000,4.000000,2.000000,3.000000,4563.500000,1290.000000,3375.500000,0.732500,3908.000000,67.000000,0.702000,0.184000,1.000000,0.000000
75%,52.000000,3.000000,3.000000,2.000000,0.000000,40.00000,5.000000,3.000000,3.000000,10707.500000,1793.000000,9720.000000,0.859750,4750.750000,80.000000,0.818000,0.521000,1.000000,1.000000
max,73.000000,5.000000,5.000000,4.000000,3.000000,56.00000,6.000000,6.000000,6.000000,34516.000000,2517.000000,34516.000000,3.397000,16920.000000,131.000000,3.250000,0.992000,1.000000,1.000000


In [26]:
predictions["Education_Level"].fillna(predictions["Education_Level"].argmax(), inplace=True)
predictions["Income_Category"].fillna(predictions["Income_Category"].argmax(), inplace=True)
predictions.head()


,Customer_Age,Dependent_count,Education_Level,Income_Category,Card_Category,Months_on_book,Total_Relationship_Count,Months_Inactive_12_mon,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio,F,M
0,48,3,0.0,0.0,0.0,39,4,3,4,2991.0,1508,1483.0,0.703,3734,64,0.882,0.504,1,0
1,59,1,0.0,0.0,0.0,53,5,5,4,2192.0,1569,623.0,0.706,4010,79,0.717,0.716,0,1
2,37,2,3.0,0.0,0.0,36,4,3,3,1734.0,987,747.0,0.879,4727,67,0.914,0.569,1,0
3,47,3,5.0,1.0,0.0,36,4,2,3,4786.0,1516,3270.0,0.940,4973,74,0.850,0.317,0,1
4,42,3,3.0,2.0,0.0,33,3,3,2,3714.0,2170,1544.0,0.524,1454,35,0.522,0.584,0,1


In [27]:
y_final = rf.predict(predictions)
y_final[:5]


array([1, 1, 1, 1, 1], dtype=int64)

In [28]:
y_final = pd.DataFrame(y_final)
y_final.rename(columns={0: "target"}, inplace=True)
y_final.head()


,target
0,1
1,1
2,1
3,1
4,1


In [29]:
y_final.to_json("predictions.json")


In [30]:
# Prediction results:
#     Only using test:
#         Argmax: 0.903338
#         Mean:   0.901089
#     Using full train:
#         Argmax: 0.915479
#         Mean:   0.914624
